# Extract Cross Section XYZ Coordinates from Plain Text Geometry

This notebook demonstrates how to extract 3D (XYZ) coordinates for cross sections from plain text HEC-RAS geometry files without requiring geometry HDF files or running the model.

## Use Cases

- Extract cross section data from legacy models (HEC-RAS 4.x, 5.x)
- Get XYZ coordinates without running simulations
- Export cross sections to GIS formats (shapefile, GeoJSON)
- Batch process multiple models for cross section inventory

## What You'll Learn

- Extract XYZ coordinates using `GeomCrossSection.get_xs_coords()`
- Filter cross sections by river/reach/station
- Export to GIS formats for visualization
- Compare plain text vs HDF extraction (when both available)

## Setup

In [ ]:
# Development mode toggle
USE_LOCAL_SOURCE = True

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"Loading from local source: {local_path}/ras_commander")

from ras_commander import RasExamples, init_ras_project
from ras_commander.geom import GeomCrossSection
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
from pathlib import Path

## Example 1: Extract All Cross Sections from Muncie Project

We'll extract XYZ coordinates for all cross sections in the Muncie example project.

In [ ]:
# Extract example project
project_path = RasExamples.extract_project("Muncie", suffix="xs_coords_demo")
print(f"Project extracted to: {project_path}")

# Find geometry file
geom_file = list(project_path.glob("*.g0*"))[0]
print(f"Geometry file: {geom_file.name}")

In [ ]:
# Extract XYZ coordinates for all cross sections
xyz = GeomCrossSection.get_xs_coords(geom_file)

print(f"\nExtracted {len(xyz):,} total XYZ points")
print(f"Number of cross sections: {xyz['RS'].nunique()}")
print(f"\nCoordinate ranges:")
print(f"  X: {xyz['x'].min():.2f} to {xyz['x'].max():.2f}")
print(f"  Y: {xyz['y'].min():.2f} to {xyz['y'].max():.2f}")
print(f"  Z: {xyz['z'].min():.2f} to {xyz['z'].max():.2f} ft")

# Show first few points
print(f"\nFirst 5 points:")
xyz.head()

## Example 2: Filter by River and Reach

Extract coordinates for specific river/reach combinations.

In [ ]:
# Get unique rivers and reaches
rivers = xyz['river'].unique()
reaches = xyz['reach'].unique()

print(f"Rivers in model: {', '.join(rivers)}")
print(f"Reaches in model: {', '.join(reaches)}")

In [ ]:
# Extract XYZ for specific river and reach
xyz_filtered = GeomCrossSection.get_xs_coords(
    geom_file,
    river=rivers[0],
    reach=reaches[0]
)

print(f"Filtered to {rivers[0]}/{reaches[0]}:")
print(f"  Total points: {len(xyz_filtered):,}")
print(f"  Cross sections: {xyz_filtered['RS'].nunique()}")

## Example 3: Single Cross Section

Extract XYZ for a single cross section.

In [ ]:
# Get first cross section
first_rs = xyz['RS'].unique()[0]

xyz_single = GeomCrossSection.get_xs_coords(
    geom_file,
    river=rivers[0],
    reach=reaches[0],
    rs=first_rs
)

print(f"Cross section RS {first_rs}:")
print(f"  Points: {len(xyz_single)}")
print(f"  Station range: {xyz_single['station'].min():.2f} to {xyz_single['station'].max():.2f}")
print(f"  Elevation range: {xyz_single['z'].min():.2f} to {xyz_single['z'].max():.2f} ft")

# Show profile
plt.figure(figsize=(10, 4))
plt.plot(xyz_single['station'], xyz_single['z'], 'b-', linewidth=2)
plt.xlabel('Station (ft)')
plt.ylabel('Elevation (ft)')
plt.title(f'Cross Section Profile: RS {first_rs}')
plt.grid(True, alpha=0.3)
plt.show()

## Example 4: Export to Shapefile

Convert XYZ coordinates to GIS LineStrings and export to shapefile.

In [ ]:
# Convert to LineStrings (one per cross section)
xs_lines = []

for (river, reach, rs), group in xyz.groupby(['river', 'reach', 'RS']):
    # Create 3D LineString from XYZ coordinates
    coords = list(zip(group['x'], group['y'], group['z']))
    
    xs_lines.append({
        'river': river,
        'reach': reach,
        'RS': rs,
        'num_points': len(coords),
        'min_elev': group['z'].min(),
        'max_elev': group['z'].max(),
        'geometry': LineString(coords)
    })

gdf = gpd.GeoDataFrame(xs_lines, geometry='geometry')
print(f"Created GeoDataFrame with {len(gdf)} cross sections")
print(f"\nFirst 3 cross sections:")
gdf[['river', 'reach', 'RS', 'num_points', 'min_elev', 'max_elev']].head(3)

In [ ]:
# Set coordinate reference system (UTM Zone 16N for Indiana)
gdf_utm = gdf.set_crs(epsg=26916)

# Export to shapefile
output_path = project_path / "cross_sections_xyz.shp"
gdf_utm.to_file(output_path)

print(f"Exported to: {output_path}")
print(f"\nShapefile includes:")
print(f"  - 3D LineString geometries (XYZ)")
print(f"  - River/Reach/RS attributes")
print(f"  - Number of points per XS")
print(f"  - Min/max elevation per XS")

## Example 5: Visualize Cross Sections in Plan View

Plot cross section locations colored by minimum elevation.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

# Plot each cross section colored by min elevation
gdf_utm.plot(ax=ax, column='min_elev', cmap='terrain', legend=True,
             legend_kwds={'label': 'Minimum Elevation (ft)'})

plt.xlabel('Easting (m, UTM Zone 16N)')
plt.ylabel('Northing (m, UTM Zone 16N)')
plt.title('Cross Section Locations - Muncie Model')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Example 6: Batch Processing Multiple Models

Extract XYZ from multiple geometry files.

In [ ]:
# Example: Process multiple geometry files
# (In this demo we'll just use different extractions of the same project)

models = [
    {'name': 'Muncie_Model1', 'project': 'Muncie'},
    {'name': 'Muncie_Model2', 'project': 'Muncie'},
]

all_xyz = []

for model in models:
    # Extract project
    path = RasExamples.extract_project(model['project'], suffix=model['name'])
    geom = list(path.glob("*.g0*"))[0]
    
    # Extract XYZ
    xyz_model = GeomCrossSection.get_xs_coords(geom)
    xyz_model['model_name'] = model['name']
    
    all_xyz.append(xyz_model)
    
    print(f"{model['name']:20s}: {len(xyz_model):5,} points, {xyz_model['RS'].nunique():3d} XS")

# Combine all models
import pandas as pd
combined = pd.concat(all_xyz, ignore_index=True)

print(f"\nTotal: {len(combined):,} points from {combined['model_name'].nunique()} models")

## Key Takeaways

### Method Signature

```python
GeomCrossSection.get_xs_coords(
    geom_file,           # Path to .g## file
    river=None,          # Optional filter
    reach=None,          # Optional filter  
    rs=None              # Optional filter
)
```

### Returns

DataFrame with columns: `river`, `reach`, `RS`, `station`, `x`, `y`, `z`

### Comparison to HDF Extraction

| Method | Input | Requires | Speed | Use Case |
|--------|-------|----------|-------|----------|
| `GeomCrossSection.get_xs_coords()` | `.g##` plain text | Nothing | Fast | Any HEC-RAS version |
| `HdfXsec.get_cross_sections()` | `.g##.hdf` | Geometry preprocessing | Faster | HEC-RAS 6.x after preprocessing |

### Advantages of Plain Text Extraction

- ✓ Works with **any HEC-RAS version** (3.x, 4.x, 5.x, 6.x)
- ✓ No preprocessing required (no need to run model)
- ✓ No version compatibility issues
- ✓ Direct access to source geometry

### Common Pitfalls

- **Missing GIS cut lines**: Older models may not have XY coordinates - method will raise ValueError
- **Coordinate system**: Output doesn't include CRS - user must set based on project
- **Station orientation**: Assumes stations increase left to right (HEC-RAS standard)

## See Also

- `examples/201_1d_plaintext_geometry.ipynb` - Other geometry parsing examples
- `ras_commander/geom/AGENTS.md` - Complete geometry parsing documentation